# Analizziamo l'effetto Memoria dei mercati prima mediante semplice correlazione e poi scomponendo i setup:

Partiamo da una serie Daily:

In [1]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "AAPL_Daily.txt"
startDate = "20130810"
endDate = "20180810"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Vol'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)
instrument['DeltaAvgPricePerc'] = round((instrument['AvgPrice'] - instrument['AvgPrice'].shift(1)) / instrument['AvgPrice'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
instrument.head(10)

5.4.0


Il file contiene 4682 record


,Open,High,Low,Close,Vol,AvgPrice,Range,Body,CloseOpenPerc,BodyRangePerc,CloseBarPerc,DeltaClosePerc,DeltaAvgPricePerc
Date_Time,,,,,,,,,,,,,
2013-08-12 16:00:00,65.27,66.95,65.23,66.77,91108290,66.05,1.72,1.50,2.30,87.21,89.53,2.85,1.21
2013-08-13 16:00:00,67.28,70.67,66.86,69.94,220484936,68.69,3.81,2.66,3.95,69.82,80.84,4.75,4.00
2013-08-14 16:00:00,71.13,72.04,70.49,71.21,189092911,71.22,1.55,0.08,0.11,5.16,46.45,1.82,3.68
2013-08-15 16:00:00,70.92,71.77,69.87,71.13,122573507,70.92,1.90,0.21,0.30,11.05,66.32,-0.11,-0.42
2013-08-16 16:00:00,71.45,71.85,71.27,71.76,90575856,71.58,0.58,0.31,0.43,53.45,84.48,0.89,0.93
2013-08-19 16:00:00,72.05,73.39,72.00,72.53,127629481,72.49,1.39,0.48,0.67,34.53,38.13,1.07,1.27
2013-08-20 16:00:00,72.82,72.94,71.55,71.58,89671743,72.22,1.39,1.24,-1.70,89.21,2.16,-1.31,-0.37
2013-08-21 16:00:00,71.94,72.45,71.60,71.77,83969305,71.94,0.85,0.17,-0.24,20.00,20.00,0.27,-0.39
2013-08-22 16:00:00,72.14,72.23,71.17,71.85,61051676,71.85,1.06,0.29,-0.40,27.36,64.15,0.11,-0.13


### Cosa NON FARE: cercare correlazioni tra le serie grezze dei prezzi!

In [2]:
for i in range(25):
    print(instrument["AvgPrice"][i], 
          instrument["AvgPrice"].shift(1)[i], 
          instrument["AvgPrice"].shift(2)[i],
          instrument["AvgPrice"].shift(3)[i],
          instrument["AvgPrice"].shift(4)[i],
          instrument["AvgPrice"].shift(5)[i])

66.05 nan nan nan nan nan
68.69 66.05 nan nan nan nan
71.22 68.69 66.05 nan nan nan
70.92 71.22 68.69 66.05 nan nan
71.58 70.92 71.22 68.69 66.05 nan
72.49 71.58 70.92 71.22 68.69 66.05
72.22 72.49 71.58 70.92 71.22 68.69
71.94 72.22 72.49 71.58 70.92 71.22
71.85 71.94 72.22 72.49 71.58 70.92
71.68 71.85 71.94 72.22 72.49 71.58
71.94 71.68 71.85 71.94 72.22 72.49
70.55 71.94 71.68 71.85 71.94 72.22
69.96 70.55 71.94 71.68 71.85 71.94
70.39 69.96 70.55 71.94 71.68 71.85
69.95 70.39 69.96 70.55 71.94 71.68
70.21 69.95 70.39 69.96 70.55 71.94
71.32 70.21 69.95 70.39 69.96 70.55
71.06 71.32 70.21 69.95 70.39 69.96
70.93 71.06 71.32 70.21 69.95 70.39
72.24 70.93 71.06 71.32 70.21 69.95
71.35 72.24 70.93 71.06 71.32 70.21
66.9 71.35 72.24 70.93 71.06 71.32
67.24 66.9 71.35 72.24 70.93 71.06
66.81 67.24 66.9 71.35 72.24 70.93
65.0 66.81 67.24 66.9 71.35 72.24


In [3]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.AvgPrice.corr(instrument.AvgPrice.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="orange", title="Correlations between AvgPrices")

Le eventuali correlazioni sono "annacquate" dal valore nominale del prezzo che porta a valori prossimi al 100%!

### Effetto Memoria

In [4]:
for i in range(25):
    print(instrument["Range"][i], 
          instrument["Range"].shift(1)[i], 
          instrument["Range"].shift(2)[i],
          instrument["Range"].shift(3)[i],
          instrument["Range"].shift(4)[i],
          instrument["Range"].shift(5)[i])

1.72 nan nan nan nan nan
3.81 1.72 nan nan nan nan
1.55 3.81 1.72 nan nan nan
1.9 1.55 3.81 1.72 nan nan
0.58 1.9 1.55 3.81 1.72 nan
1.39 0.58 1.9 1.55 3.81 1.72
1.39 1.39 0.58 1.9 1.55 3.81
0.85 1.39 1.39 0.58 1.9 1.55
1.06 0.85 1.39 1.39 0.58 1.9
0.57 1.06 0.85 1.39 1.39 0.58
1.39 0.57 1.06 0.85 1.39 1.39
2.32 1.39 0.57 1.06 0.85 1.39
1.4 2.32 1.39 0.57 1.06 0.85
0.77 1.4 2.32 1.39 0.57 1.06
0.92 0.77 1.4 2.32 1.39 0.57
1.36 0.92 0.77 1.4 2.32 1.39
0.85 1.36 0.92 0.77 1.4 2.32
1.01 0.85 1.36 0.92 0.77 1.4
1.35 1.01 0.85 1.36 0.92 0.77
0.63 1.35 1.01 0.85 1.36 0.92
2.56 0.63 1.35 1.01 0.85 1.36
1.27 2.56 0.63 1.35 1.01 0.85
1.34 1.27 2.56 0.63 1.35 1.01
1.01 1.34 1.27 2.56 0.63 1.35
2.05 1.01 1.34 1.27 2.56 0.63


In [5]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.Range.corr(instrument.Range.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="orange", title="Correlations between Ranges")

In [6]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.Body.corr(instrument.Body.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="orange", title="Correlations between Bodies")

Analizziamo il medesimo effetto memoria su valori percentuali di variazione:

In [7]:
instrument["RangePerc"] = round((instrument['Range'] - instrument['Range'].shift(1)) / instrument['Range'].shift(1) * 100,2)

for i in range(25):
    print(instrument["RangePerc"][i], 
          instrument["RangePerc"].shift(1)[i], 
          instrument["RangePerc"].shift(2)[i],
          instrument["RangePerc"].shift(3)[i],
          instrument["RangePerc"].shift(4)[i],
          instrument["RangePerc"].shift(5)[i])

nan nan nan nan nan nan
121.51 nan nan nan nan nan
-59.32 121.51 nan nan nan nan
22.58 -59.32 121.51 nan nan nan
-69.47 22.58 -59.32 121.51 nan nan
139.66 -69.47 22.58 -59.32 121.51 nan
0.0 139.66 -69.47 22.58 -59.32 121.51
-38.85 0.0 139.66 -69.47 22.58 -59.32
24.71 -38.85 0.0 139.66 -69.47 22.58
-46.23 24.71 -38.85 0.0 139.66 -69.47
143.86 -46.23 24.71 -38.85 0.0 139.66
66.91 143.86 -46.23 24.71 -38.85 0.0
-39.66 66.91 143.86 -46.23 24.71 -38.85
-45.0 -39.66 66.91 143.86 -46.23 24.71
19.48 -45.0 -39.66 66.91 143.86 -46.23
47.83 19.48 -45.0 -39.66 66.91 143.86
-37.5 47.83 19.48 -45.0 -39.66 66.91
18.82 -37.5 47.83 19.48 -45.0 -39.66
33.66 18.82 -37.5 47.83 19.48 -45.0
-53.33 33.66 18.82 -37.5 47.83 19.48
306.35 -53.33 33.66 18.82 -37.5 47.83
-50.39 306.35 -53.33 33.66 18.82 -37.5
5.51 -50.39 306.35 -53.33 33.66 18.82
-24.63 5.51 -50.39 306.35 -53.33 33.66
102.97 -24.63 5.51 -50.39 306.35 -53.33


In [8]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.RangePerc.corr(instrument.RangePerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="orange", title="Correlations between RangePerc")

### Proviamo ulteriori relazioni:

In [9]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.DeltaClosePerc.corr(instrument.DeltaClosePerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="blue", title="Correlations between DeltaClosePerc")

In [10]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.CloseOpenPerc.corr(instrument.CloseOpenPerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="red", title="Correlations between CloseOpenPerc")

In [11]:
for i in range(25):
    print(instrument["BodyRangePerc"][i], 
          instrument["BodyRangePerc"].shift(1)[i], 
          instrument["BodyRangePerc"].shift(2)[i],
          instrument["BodyRangePerc"].shift(3)[i],
          instrument["BodyRangePerc"].shift(4)[i],
          instrument["BodyRangePerc"].shift(5)[i])

87.21 nan nan nan nan nan
69.82 87.21 nan nan nan nan
5.16 69.82 87.21 nan nan nan
11.05 5.16 69.82 87.21 nan nan
53.45 11.05 5.16 69.82 87.21 nan
34.53 53.45 11.05 5.16 69.82 87.21
89.21 34.53 53.45 11.05 5.16 69.82
20.0 89.21 34.53 53.45 11.05 5.16
27.36 20.0 89.21 34.53 53.45 11.05
57.89 27.36 20.0 89.21 34.53 53.45
22.3 57.89 27.36 20.0 89.21 34.53
57.76 22.3 57.89 27.36 20.0 89.21
50.0 57.76 22.3 57.89 27.36 20.0
0.0 50.0 57.76 22.3 57.89 27.36
75.0 0.0 50.0 57.76 22.3 57.89
47.06 75.0 0.0 50.0 57.76 22.3
15.29 47.06 75.0 0.0 50.0 57.76
70.3 15.29 47.06 75.0 0.0 50.0
2.96 70.3 15.29 47.06 75.0 0.0
26.98 2.96 70.3 15.29 47.06 75.0
64.45 26.98 2.96 70.3 15.29 47.06
7.87 64.45 26.98 2.96 70.3 15.29
44.78 7.87 64.45 26.98 2.96 70.3
63.37 44.78 7.87 64.45 26.98 2.96
76.1 63.37 44.78 7.87 64.45 26.98


In [12]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.BodyRangePerc.corr(instrument.BodyRangePerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="blue", title="Correlations between BodyRangePerc")

In [13]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.CloseOpenPerc.corr(instrument.BodyRangePerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="blue", title="Correlations between CloseOpenPerc")

In [14]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.CloseBarPerc.corr(instrument.BodyRangePerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="blue", title="Correlations between CloseBarPerc")

In [15]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(instrument.DeltaAvgPricePerc.corr(instrument.DeltaAvgPricePerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="blue", title="Correlations between DeltaAvgPricePerc")

## Ma come pesare tali valori apparentemente piccoli? Utilizziamo un approccio quantitativo costruito su setup condizionali introducendo il tema dei Trading System:

In [16]:
rule = (instrument["Close"].shift(1) < instrument["Low"].shift(2)) & \
       (instrument["Close"].shift(2) < instrument["Low"].shift(3)) & \
       (instrument["Close"].shift(3) < instrument["Low"].shift(4))
rule[90:110]

Date_Time
2013-12-18 16:00:00    False
2013-12-19 16:00:00    False
2013-12-20 16:00:00    False
2013-12-23 16:00:00    False
2013-12-24 16:00:00    False
2013-12-26 16:00:00    False
2013-12-27 16:00:00    False
2013-12-30 16:00:00    False
2013-12-31 16:00:00     True
2014-01-02 16:00:00    False
2014-01-03 16:00:00    False
2014-01-06 16:00:00    False
2014-01-07 16:00:00    False
2014-01-08 16:00:00    False
2014-01-09 16:00:00    False
2014-01-10 16:00:00    False
2014-01-13 16:00:00    False
2014-01-14 16:00:00    False
2014-01-15 16:00:00    False
2014-01-16 16:00:00    False
dtype: bool

Equivalente a (notazione ad oggetti che riprenderemo nel Modulo 4 dedicato ai Trading System):

In [17]:
rule = (instrument.Close.shift(1) < instrument.Low.shift(2)) & \
       (instrument.Close.shift(2) < instrument.Low.shift(3)) & \
       (instrument.Close.shift(3) < instrument.Low.shift(4))
rule[90:110]

Date_Time
2013-12-18 16:00:00    False
2013-12-19 16:00:00    False
2013-12-20 16:00:00    False
2013-12-23 16:00:00    False
2013-12-24 16:00:00    False
2013-12-26 16:00:00    False
2013-12-27 16:00:00    False
2013-12-30 16:00:00    False
2013-12-31 16:00:00     True
2014-01-02 16:00:00    False
2014-01-03 16:00:00    False
2014-01-06 16:00:00    False
2014-01-07 16:00:00    False
2014-01-08 16:00:00    False
2014-01-09 16:00:00    False
2014-01-10 16:00:00    False
2014-01-13 16:00:00    False
2014-01-14 16:00:00    False
2014-01-15 16:00:00    False
2014-01-16 16:00:00    False
dtype: bool

Di fatto abbiamo isolato tutti i record la cui chiusura precedente sia stata inferiore al minimo del record precedente per tre volte consecutive!

In [18]:
instrument["Rule"] = rule
#instrument["RuleDecoded"] = instrument.Rule.apply(lambda x: 1 if x == True else 0)
instrument['RuleDecoded'] = np.where(instrument['Rule'] == True, 1, 0)
#instrument["CloseOpenPercDecoded"] = instrument["CloseOpenPerc"].apply(lambda x: 1 if x > 0 else 0)
instrument['CloseOpenPercDecoded'] = np.where(instrument['CloseOpenPerc'] > 0, 1, 0)
instrument[90:110]

,Open,High,Low,Close,Vol,AvgPrice,Range,Body,CloseOpenPerc,BodyRangePerc,CloseBarPerc,DeltaClosePerc,DeltaAvgPricePerc,RangePerc,Rule,RuleDecoded,CloseOpenPercDecoded
Date_Time,,,,,,,,,,,,,,,,,
2013-12-18 16:00:00,78.53,78.78,76.97,78.68,141465807,78.24,1.81,0.15,0.19,8.29,94.48,-0.76,-1.47,108.05,False,0,1
2013-12-19 16:00:00,78.50,78.57,77.68,77.78,80239369,78.13,0.89,0.72,-0.92,80.90,11.24,-1.14,-0.14,-50.83,False,0,0
2013-12-20 16:00:00,77.92,78.80,77.83,78.43,109103435,78.24,0.97,0.51,0.65,52.58,61.86,0.84,0.14,8.99,False,0,1
2013-12-23 16:00:00,81.14,81.53,80.39,81.44,125326831,81.12,1.14,0.30,0.37,26.32,92.11,3.84,3.68,17.53,False,0,1
2013-12-24 16:00:00,81.41,81.70,80.86,81.10,42002016,81.27,0.84,0.31,-0.38,36.90,28.57,-0.42,0.18,-26.32,False,0,0
2013-12-26 16:00:00,81.16,81.36,80.48,80.56,51002035,80.89,0.88,0.60,-0.74,68.18,9.09,-0.67,-0.47,4.76,False,0,0
2013-12-27 16:00:00,80.55,80.63,79.93,80.01,56471317,80.28,0.70,0.54,-0.67,77.14,11.43,-0.68,-0.75,-20.45,False,0,0
2013-12-30 16:00:00,79.71,80.01,78.90,79.22,63407722,79.46,1.11,0.49,-0.61,44.14,28.83,-0.99,-1.02,58.57,False,0,0
2013-12-31 16:00:00,79.17,80.18,79.14,80.15,55819372,79.66,1.04,0.98,1.24,94.23,97.12,1.17,0.25,-6.31,True,1,1


Contiamo i record in cui il setup sia verificato:

In [19]:
events = instrument[instrument.Rule == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!
events

29

Filtriamo ulteriormente i record in base a setup verificato e body positivo (possibile guadagno):

In [20]:
events_OK = instrument.Open[(rule==True) & (instrument.CloseOpenPerc > 0)].count()
events_OK

18

Calcoliamo la percentuale degli eventi che potenzialmente avrebbero prodotto guadagni rispetto al totale degli eventi con setup attivato:

In [21]:
percent_win = round(events_OK / events * 100,2)
percent_win

62.07

### Analisi Binaria dei risultati

In [22]:
instrument.iloc[:,-2:][75:110]

,RuleDecoded,CloseOpenPercDecoded
Date_Time,,
2013-11-26 16:00:00,0,1
2013-11-27 16:00:00,0,1
2013-11-29 16:00:00,0,1
2013-12-02 16:00:00,0,0
2013-12-03 16:00:00,0,1
2013-12-04 16:00:00,0,0
2013-12-05 16:00:00,0,0
2013-12-06 16:00:00,0,0
2013-12-09 16:00:00,0,1


In [23]:
a = instrument[(instrument.RuleDecoded == instrument.CloseOpenPercDecoded)].Open.count()
a

609

In [24]:
b = instrument.Open.count()
b

1259

In [25]:
a / b

0.4837172359015091

In [26]:
c = instrument[(instrument.RuleDecoded == 1) & (instrument.CloseOpenPercDecoded == 1)].Open.count()
c

18

In [27]:
d = instrument[(instrument.RuleDecoded == 1)].Open.count()
d

29

In [28]:
c / d

0.6206896551724138

# Codice Unico (Unendo analisi su fronte long e fronte short)

In [29]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "AAPL_Daily.txt"
startDate = "20130810"
endDate = "20180810"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Vol'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)
instrument['DeltaAvgPricePerc'] = round((instrument['AvgPrice'] - instrument['AvgPrice'].shift(1)) / instrument['AvgPrice'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)


# ANALISI LONG

rule_long = (instrument.Close.shift(1) < instrument.Low.shift(2)) & \
            (instrument.Close.shift(2) < instrument.Low.shift(3)) & \
            (instrument.Close.shift(3) < instrument.Low.shift(4))

instrument["RuleLong"] = rule_long
instrument['RuleLongDecoded'] = np.where(instrument['RuleLong'] == True, 1, 0)

instrument['CloseOpenPercDecoded_Long'] = np.where(instrument['CloseOpenPerc'] > 0, 1, 0)

events_long = instrument[instrument.RuleLong == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_long_OK = instrument.Open[(rule_long==True) & (instrument.CloseOpenPerc > 0)].count()

percent_win_long = round(events_long_OK / events_long * 100,2)

print("")
print("Regola a segno sul fronte Long per il", percent_win_long, "% dei campioni (", events_long_OK, "su", events_long,")")


# ANALISI SHORT

rule_short = (instrument.Close.shift(1) > instrument.High.shift(2)) & \
             (instrument.Close.shift(2) > instrument.High.shift(3)) & \
             (instrument.Close.shift(3) > instrument.High.shift(4))

instrument["RuleShort"] = rule_short
instrument['RuleShortDecoded'] = np.where(instrument['RuleShort'] == True, 1, 0)

instrument['CloseOpenPercDecoded_Short'] = np.where(instrument['CloseOpenPerc'] < 0, 1, 0)

events_short = instrument[instrument.RuleShort == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_short_OK = instrument.Open[(rule_short==True) & (instrument.CloseOpenPerc < 0)].count()

percent_win_short = round(events_short_OK / events_short * 100,2)

print("")
print("Regola a segno sul fronte Short per il", percent_win_short, "% dei campioni(", events_short_OK, "su", events_short,")")

5.4.0


Il file contiene 4682 record

Regola a segno sul fronte Long per il 62.07 % dei campioni ( 18 su 29 )

Regola a segno sul fronte Short per il 40.91 % dei campioni( 18 su 44 )
